In [ ]:
import sys
import json
import pandas as pd
import joblib
import numpy as np
import warnings
# Suppress user warnings related to LightGBM or scikit-learn
warnings.filterwarnings('ignore')

def predict():
    """
    Loads the trained LightGBM model and predicts click probabilities 
    for listings based on features passed via stdin.
    """
    
    # 1. Define the features (MUST match the training script's order)
    FEATURE_COLS = [
        'wilaya_match', 'quality_score', 'price_dzd', 'price_per_kg',
        'available_quantity_kg', 'seller_rating', 'high_seller_rating', 
        'is_fresh_listing'
    ]
    
    try:
        # Read all JSON data sent from Node.js via STDIN
        input_data_json = sys.stdin.read()
        
        # Check if any data was sent
        if not input_data_json:
            # Output an empty list of scores if no data is provided
            print(json.dumps([]))
            return
            
        # Parse the JSON array of feature objects
        data_to_predict = json.loads(input_data_json)
        
    except Exception as e:
        # Output an error to stderr for Node.js debugging
        sys.stderr.write(f"Error reading or parsing input data: {e}\n")
        # Output an empty list to stdout so Node.js JSON parsing doesn't fail
        print(json.dumps([]))
        return

    try:
        # 2. Load the trained model
        MODEL_PATH = 'recommendation_model_lgbm.pkl'
        model = joblib.load(MODEL_PATH)
        
        # 3. Create the Pandas DataFrame for prediction
        X_predict = pd.DataFrame(data_to_predict)
        
        # CRITICAL: Ensure columns are in the EXACT SAME ORDER as used during training
        # The model expects its features in a specific sequence.
        X_predict = X_predict[FEATURE_COLS]
        
        # 4. Predict the probability of a click (the ranking score)
        # We only need the probability for the positive class (index 1)
        probabilities = model.predict_proba(X_predict)[:, 1]
        
        # 5. Output the result to STDOUT as a JSON array
        # Node.js will read this output.
        print(json.dumps(probabilities.tolist()))

    except Exception as e:
        sys.stderr.write(f"Prediction or model loading error: {e}\n")
        print(json.dumps([])) # Fail gracefully by sending empty list
        
if __name__ == '__main__':
    predict()